## Treinamento com Transfer Learning e MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np
import os
import re

train_dir = 'dataset_aumentado/train'
val_dir = 'dataset_aumentado/val'
test_dir = 'dataset_aumentado/test'

train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=16, class_mode='binary'
)
val_data = val_gen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=16, class_mode='binary'
)
test_data = test_gen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=16, class_mode='binary', shuffle=False
)

result_file = 'resultados_20_execucoes2.txt'
if os.path.exists(result_file):
    os.remove(result_file)

best_f1 = 0

# Listas para históricos e matrizes de confusão
all_acc = []
all_val_acc = []
all_loss = []
all_val_loss = []
all_conf_matrices = []

print("\n✅ Iniciando as 20 execuções...\n")

# ========================
# Execuções (Treino 20x)
# ========================
for exec_num in range(1, 21):
    print(f"\n========== Execução {exec_num} ==========\n")

    # Modelo
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    preds = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=preds)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Class weights
    total_train = train_data.classes.shape[0]
    class_0 = np.sum(train_data.classes == 0)
    class_1 = np.sum(train_data.classes == 1)
    weight_for_0 = (1 / class_0) * (total_train / 2.0)
    weight_for_1 = (1 / class_1) * (total_train / 2.0)
    class_weights = {0: weight_for_0, 1: weight_for_1}

    # Treinamento
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=20,
        class_weight=class_weights,
        verbose=2
    )

    all_acc.append(history.history['accuracy'])
    all_val_acc.append(history.history['val_accuracy'])
    all_loss.append(history.history['loss'])
    all_val_loss.append(history.history['val_loss'])

    # Avaliação
    loss, acc = model.evaluate(test_data, verbose=0)
    y_true = test_data.classes
    y_pred_probs = model.predict(test_data, verbose=0).flatten()
    y_pred = (y_pred_probs > 0.5).astype(int)

    report = classification_report(y_true, y_pred, target_names=['nao_poluido', 'poluido'])
    conf_matrix = confusion_matrix(y_true, y_pred)
    all_conf_matrices.append(conf_matrix)
    f1 = f1_score(y_true, y_pred)

    # Salvar resultados da execução no txt
    with open(result_file, 'a') as f:
        f.write(f'\n========== Execução {exec_num} ==========\n')
        f.write(f'Train Accuracy: {history.history["accuracy"][-1]:.4f}\n')
        f.write(f'Train Loss: {history.history["loss"][-1]:.4f}\n')
        f.write(f'Val Accuracy: {history.history["val_accuracy"][-1]:.4f}\n')
        f.write(f'Val Loss: {history.history["val_loss"][-1]:.4f}\n')
        f.write(f'Test Accuracy: {acc:.4f}\n')
        f.write(f'F1 no Teste: {f1:.4f}\n')
        f.write('\nClassification Report:\n')
        f.write(report)
        f.write('\nConfusion Matrix:\n')
        f.write(np.array2string(conf_matrix))
        f.write('\n\n')

    # Salvar melhor modelo
    if f1 > best_f1:
        best_f1 = f1
        model.save('melhor_modelo.h5')
        print(f'✅ Novo melhor modelo salvo com F1: {best_f1:.4f}')

# Após o loop, salve os históricos e matrizes em arquivos .npy
np.save('all_acc.npy', np.array(all_acc))
np.save('all_val_acc.npy', np.array(all_val_acc))
np.save('all_loss.npy', np.array(all_loss))
np.save('all_val_loss.npy', np.array(all_val_loss))
np.save('all_conf_matrices.npy', np.array(all_conf_matrices))

print("\n✅ As 20 execuções terminaram. Resultados salvos no arquivo:", result_file)
print("✅ Históricos de treino e matrizes de confusão salvos em arquivos .npy")

## Calculos dos valores médios obtidos

In [ ]:
# ==============================================
# Parte 2 - Cálculo das médias das 20 execuções
# ==============================================
metrics_to_average = ['Train Accuracy', 'Train Loss', 'Val Accuracy', 'Val Loss', 'Test Accuracy', 'F1 no Teste']
results = {metric: [] for metric in metrics_to_average}

with open(result_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        for metric in metrics_to_average:
            if line.startswith(metric):
                value = float(re.findall(r"[-+]?\d*\.\d+|\d+", line.strip().split(':')[-1])[0])
                results[metric].append(value)

summary_file = 'resumo_final.txt'
with open(summary_file, 'w') as f:
    f.write("Médias das 20 Execuções:\n\n")
    for metric in metrics_to_average:
        values = results[metric]
        mean = np.mean(values)
        std = np.std(values)
        f.write(f'{metric}: Média = {mean:.4f} | Desvio Padrão = {std:.4f}\n')

print("\n✅ Resumo final com as médias salvo em:", summary_file)

## BoxPlot

In [ ]:
import matplotlib.pyplot as plt

# Supondo que você já tem o dicionário 'results' preenchido
plt.figure(figsize=(10,6))
plt.boxplot([results[m] for m in metrics_to_average], labels=metrics_to_average, showmeans=True)
plt.title('Distribuição das métricas nas 20 execuções')
plt.ylabel('Valor')
plt.grid(True)
plt.show()